# Organization

I'm going to split the master code up top into manageable code blocks.  Let's start with import statements

In [1]:
########################## Import Statements #################################
try:
    import PySimpleGUI as sg
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import random
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import networkx as nx
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import numpy
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import csv
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import re
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import math
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")
try:
    import seaborn as sns
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import numpy as np
except ModuleNotFoundError as e:
    print("You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

try:
    import scipy.stats as st
except ModuleNotFoundError as e:
    print(f"You are missing a necessary package.  Install the package by copying and pasting the below into your command window")
    print(f"pip install {e.name}")

# The parts that make the Data Input Tab Work

In [2]:
########################## Functions to make the math work ###################

def network_from_xy_csv(input_filename,connection_radius):
    """Using input_filename as the input, returns a NetworkX graph object G, 
        as well as the lists of coordinates (one featuring all X values, the other featuring Y).
        Nodes are generated based on the number of points, and arcs are generated based on the 
        connection radius of nodes. Any nodes that do not connect to the main body are dropped.
        This function uses matplotlib to plot the raw data geographically, and provides an 
        additional plot of the data showing the arcs created. Requires importation of NetworkX 
        as nx, matplotlib.pyplot as plt, and csv.
        
        Args:
        input_filename(CSV): The file to access the X and Y coordinates from. CSV file should
            be structured as follows:
            "x","y",
            x[0],y[0],
            x[1],y[1],
            .
            .
            .
        connection_radius(int): The radius around each node in which it will form arcs with
            surrounding nodes.
        
        Returns:
        G(nx.Graph object)
        X(List): X coordinates
        Y(List): Y coordinates
    """
    # This block creates X and Y coordinate lists by interpreting the CSV file.
    sensor_X_coords = []
    sensor_Y_coords = []
    with open(input_filename) as f:
        inreader = csv.reader(f)
        inreader = list(inreader)
        with open(input_filename) as f:
            inreader = csv.reader(f)
            inreader = list(inreader)
            if 'x' in inreader[0]:
                x_index = inreader[0].index('x')
            if 'X' in inreader[0]:
                x_index = inreader[0].index('X')
            if 'y' in inreader[0]:
                y_index = inreader[0].index('y')
            if 'Y' in inreader[0]:
                y_index = inreader[0].index('Y')
            for line in range(1,len(inreader)):
                sensor_X_coords.append(float(inreader[line][x_index]))
                sensor_Y_coords.append(float(inreader[line][y_index]))
                
    # This block creates the two plots and initializes graph object using the coordinate lists.
    print('Total sensors deployed: ',len(sensor_X_coords))
    fig, axes = plt.subplots(1,2)
    
    G = nx.Graph()
    G.add_nodes_from(range(len(sensor_X_coords)))
    
    # This block creates and adds arcs based on connection radius
    for i in range(len(sensor_X_coords)):    
        for j in range(len(sensor_X_coords)):
            if (math.sqrt((sensor_X_coords[i ]- sensor_X_coords[j])**2 + (sensor_Y_coords[i] - sensor_Y_coords[j])**2)) <= connection_radius and (i != j):
                G.add_edge(i,j)
                line_start = [sensor_X_coords[i], sensor_X_coords[j]]
                line_end = [sensor_Y_coords[i], sensor_Y_coords[j]]
                axes[1].plot(line_start,line_end,color='gray',linestyle='dashed',zorder=1)
    
    # This block drops any nodes that are unconnected(have a degree of 0)
    for i in list(G.nodes):
        if G.degree(i) == 0:
            j = list(G.nodes).index(i)
            G.remove_node(i)
            del sensor_X_coords[j]
            del sensor_Y_coords[j]
    print('Surviving Nodes: ',len(list(G.nodes)))
    
    axes[0].scatter(sensor_X_coords,sensor_Y_coords,zorder=2)
    axes[1].scatter(sensor_X_coords,sensor_Y_coords,zorder=2)
    axes[0].set_title('Initial Plot of Sensors')
    axes[0].set_ylabel('Y')
    axes[0].set_xlabel('X')
    axes[1].set_title('Sensors Connected')
    axes[1].set_ylabel('Y')
    axes[1].set_xlabel('X')
    fig.tight_layout()

    
    figs = plt.gcf()   #added this to extract the figure out of this function
    
    return G, sensor_X_coords, sensor_Y_coords, figs  #added figs to the return of this function




# Parts that make the report work

In [3]:
def text_report(G):
    """
    Generates the Network Statistics Report
    
    Inputs: 
        G(nx.Graph() Object):  The graph about which to report statistics
        
    Returns:
        Figs (matplotlib.figure) : The figure to be drawn to the GUI as the report
    """
    fig, ax = plt.subplots()
    fig.text(0.15, .8, 'REPORT')
    fig.text(0.15, .6, f'You successfully launched and connected {G.order()} sensors')
    fig.text(0.15, .5, f'Your network has {G.size()} sensor connections')
    fig.text(0.15, .4, f'Your network average sensor connections is {round(2*G.size()/G.order(),3)}')
    fig.text(0.15, .3, f'Is your network Fully Connected? : {G.order()*(G.order()-1)/2 == G.size()}')
    fig.text(0.15, .2, f'Is your network Connected? : {nx.is_connected(G)}')
    
    ax.tick_params(bottom=False,left=False,right=False,top=False)
    ax.set_xticks([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_yticklabels([])
    
    figs = plt.gcf()
    
    return figs


def text_report_reliability(G, mean, lb, ub):
    """
    Generates the Network Statistics Report
    
    Inputs: 
        G(nx.Graph() Object):  The graph about which to report statistics
        mean (float): the average day that network failure was achieved. 
        lb (float): The lower bound of the prediction interval for network failure.
        ub (float): The upper bound of the prediction interval for network failure.
        
    Returns:
        Figs(matplotlib.figure): The figure to be drawn to the GUI as the report
    """
    mean = mean
    lb = lb
    ub = ub
    
    fig, ax = plt.subplots()
    fig.text(0.15, .8, 'REPORT')
    fig.text(0.15, .6, f'Average time unitl network failure (disconnectivity): {round(mean,3)} days')
    fig.text(0.15, .5, f'Pred Interval for 95% of Networks - Lower Bound: {round(lb,3)} days')
    fig.text(0.15, .4, f'Pred Interval for 95% of Networks - Upper Bound: {round(ub,3)} days')
    
    ax.tick_params(bottom=False,left=False,right=False,top=False)
    ax.set_xticks([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_yticklabels([])
    
    figs = plt.gcf()
    
    return figs



# The parts that make the Generate Network Tab Work

In [4]:
def generate_network_from_two_points(left_point,right_point,connection_radius,
                                     deployment_rate=3,max_num_sensors=100,aircraft_speed=300):
    """Using input parameters, generates a random distribution of sensors between two 
        geographic points that simulates an aircraft-based pass. Nodes are generated
        based on number of sensors deployed, and arcs are created based on connection
        radius between sensors.This function uses matplotlib to plot the raw data 
        geographically, and provides an additional plot of the data showing the arcs 
        created. Requires importation of NetworkX as nx, matplotlib.pyplot as plt,
        random, math, and numpy. This function can only model paths from left to 
        right. If right to left required, generate pass and then transform.
        
        Args:
        left_point(list[X,Y]): Coordinate to begin pass at.
        right_point(list[X,Y]): Coordinate to end pass at.
        connection_radius(int): The radius around each node in which it will form arcs with
            surrounding nodes.
        deployment_rate(int): The average rate of sensors deployed per minute. Default of 3.
        max_num_sensors(int): The maximum number of sensors to deploy. Default of 100.
        aircraft_speed(int): Aircraft speed in kph. Default of 300.
        
        Returns:
        G(nx.Graph object)
        X(List): X coordinates
        Y(List): Y coordinates
    """
    # This block initializes the path based on inputs
    slope = (right_point[1]-left_point[1])/(right_point[0]-left_point[0]) # angle of flight path from start point
    time = math.sqrt((right_point[1]-left_point[1])**2+(right_point[0]-left_point[0])**2)/aircraft_speed # Distance/Speed
    deployment_start_point = left_point
    total_times = 0
    sensors_deployed = 0
    left_angle = math.atan(slope)+.5*math.pi
    right_angle = math.atan(slope)+1.5*math.pi
    angles = [left_angle,right_angle]
    aircraft_X = [deployment_start_point[0]]
    aircraft_Y = [deployment_start_point[1]]
    sensor_X_coords = []
    sensor_Y_coords = []
    
    # This block simulates the pass itself and deploys sensors along the path based on inter-deployment time
    while sensors_deployed < max_num_sensors:
        inter_time = random.expovariate(deployment_rate)
        if (sensors_deployed) >= max_num_sensors:    # Checks if max number has been reached
            break
        if slope > 0:
            if (aircraft_X[-1] >= right_point[0]) or (aircraft_Y[-1] >= right_point[1]):    # Checks if end point passed
                break
        if slope < 0:
            if (aircraft_X[-1] >= right_point[0]) or (aircraft_Y[-1] <= right_point[1]):    # Checks if end point passed
                break
        total_times += inter_time
        sensors_deployed += 1
        angle = math.atan(abs(slope))
        change_in_aircraft_X = (aircraft_speed/60)*inter_time*math.cos(angle)
        change_in_aircraft_Y = (slope*change_in_aircraft_X)
        aircraft_X.append(aircraft_X[-1] + change_in_aircraft_X)
        aircraft_Y.append(aircraft_Y[-1] + change_in_aircraft_Y)
        sensor_launch_magnitude = numpy.random.exponential(8)    # Arbitrary distribution for magnitudes
        launch_angle = random.choice(angles)
        sensor_X = sensor_launch_magnitude*math.cos(launch_angle) + aircraft_X[-1]
        sensor_Y = sensor_launch_magnitude*math.sin(launch_angle) + aircraft_Y[-1]
        sensor_X_coords.append(sensor_X)
        sensor_Y_coords.append(sensor_Y)
        
    # This block creates the two plots and graph object using the coordinate lists and aircraft path
    print('Total sensors deployed: ',sensors_deployed)
    fig, axes = plt.subplots(1,2)
    
    axes[0].plot(aircraft_X,aircraft_Y,color = 'black',linestyle = 'dashed',zorder=1)
    
    G = nx.Graph()
    
    # This block creates and adds arcs based on connection radius
    G.add_nodes_from(range(len(sensor_X_coords)))
    for i in range(len(sensor_X_coords)):
        for j in range(len(sensor_X_coords)):
            if (math.sqrt((sensor_X_coords[i] - sensor_X_coords[j])**2 + (sensor_Y_coords[i] - sensor_Y_coords[j])**2)) <= connection_radius and (i != j):
                G.add_edge(i,j)
                line_start = [sensor_X_coords[i],sensor_X_coords[j]]
                line_end = [sensor_Y_coords[i],sensor_Y_coords[j]]
                axes[1].plot(line_start,line_end,color='gray',linestyle='dashed',zorder=1)
    # This block drops any nodes that are unconnected(have a degree of 0)
    for i in list(G.nodes):
        if G.degree(i) == 0:
            j = list(G.nodes).index(i)
            G.remove_node(i)
            del sensor_X_coords[j]
            del sensor_Y_coords[j]
    print('Surviving Nodes: ',len(list(G.nodes)))
    
    axes[0].scatter(sensor_X_coords,sensor_Y_coords,zorder=2)
    axes[1].scatter(sensor_X_coords,sensor_Y_coords,zorder=2)
    axes[0].set_title('Initial Plot of Sensors')
    axes[0].set_ylabel('Y')
    axes[0].set_xlabel('X')
    axes[1].set_title('Sensors Connected')
    axes[1].set_ylabel('Y')
    axes[1].set_xlabel('X')
    fig.tight_layout()
    figs = plt.gcf()   #added this to extract the figure out of this function            
    
    return G, sensor_X_coords, sensor_Y_coords, figs  #added figs to the return of this function


def generate_center_out_network(nodes,connection_radius,origin=(0,0)):
    """Using input parameters, generates a random distribution of sensors at one 
        geographic point that simulates a center-out distribution. Nodes are generated
        based on number of sensors deployed, and arcs are created based on connection
        radius between sensors.This function uses matplotlib to plot the raw data 
        geographically, and provides an additional plot of the data showing the arcs 
        created. Requires importation of NetworkX as nx, matplotlib.pyplot as plt,
        random, math, and numpy. 
        
        Args:
        nodes(int): The number of nodes to distribute
        connection_radius(int): The radius around each node in which it will form arcs with
            surrounding nodes.
        
        Returns:
        G(nx.Graph object)
        X(List): X coordinates
        Y(List): Y coordinates
    """
    # This block generates coordinate lists
    sensor_X_coords = []
    sensor_Y_coords = []
    for i in range(nodes):
        random_angle = 2*math.pi*random.random()
        launch_distance = numpy.random.exponential(5)    # Arbitrary distribution
        x_coord = launch_distance*math.cos(random_angle)+origin[0]
        y_coord = launch_distance*math.sin(random_angle)+origin[1]
        sensor_X_coords.append(x_coord)
        sensor_Y_coords.append(y_coord)
        
    # This block initializes plots and graph object using coordinate lists
    print('Total sensors deployed: ',len(sensor_X_coords))
    fig, axes = plt.subplots(1,2)

    G = nx.Graph()
    G.add_nodes_from(range(len(sensor_X_coords)))
    
    # This block creates and adds arcs based on connection radius
    for i in range(len(sensor_X_coords)):
        for j in range(len(sensor_X_coords)):
            if (math.sqrt((sensor_X_coords[i] - sensor_X_coords[j])**2 + (sensor_Y_coords[i] - sensor_Y_coords[j])**2)) <= connection_radius and (i != j):
                G.add_edge(i,j)
                line_start = [sensor_X_coords[i],sensor_X_coords[j]]
                line_end = [sensor_Y_coords[i],sensor_Y_coords[j]]
                axes[1].plot(line_start,line_end,color='gray',linestyle='dashed',zorder=1)
    
    # This block drops any nodes that are unconnected(have a degree of 0)
    for i in list(G.nodes):
        if G.degree(i) == 0:
            j = list(G.nodes).index(i)
            G.remove_node(i)
            del sensor_X_coords[j]
            del sensor_Y_coords[j]
    print('Surviving Nodes: ',len(list(G.nodes)))
    axes[0].scatter(sensor_X_coords,sensor_Y_coords,zorder=2)
    axes[1].scatter(sensor_X_coords,sensor_Y_coords,zorder=2)
    axes[0].set_title('Initial Plot of Sensors')
    axes[0].set_ylabel('Y')
    axes[0].set_xlabel('X')
    axes[1].set_title('Sensors Connected')
    axes[1].set_ylabel('Y')
    axes[1].set_xlabel('X')
    fig.tight_layout()
    figs = plt.gcf()   #added this to extract the figure out of this function

    return G, sensor_X_coords, sensor_Y_coords, figs   #added figs to the return of this function

# The parts that make the Reliability Analysis Tab Work


In [5]:
def network_degradation_test(network,survivability,days):
    """
    This is a helper function used during the implimentation of the 'average_network_durability' function. This function
    iteratively 'degrades' a network through the removal of sensors (nodes) based on a discrete time step survival probability.
    The function stops when there is either a single sensor left in the network, the network becomes disconnected, or the 
    function has iterated through each day as specified as an input; which ever occurrence happens first. This is highly
    dependent on the type of type of network constructed, the sensor survivability rate, and the number of days the network
    degrades. 
    
    For example, if the original (non-degraded) network is fully-connected, then the function can only stop once
    all but one sensor remains, or there are no longer any days to potentially degrade over.
    
    Another example; if the origninal (non-degraded) network is constructed and already disconnected, the function will stop
    at the first iteration.
    
      
    Input:
        network(nx.Graph object): The network object that the user has previously created, regardless of network 
        construction method.
        
        survibability(int with range (0-100)): Represents the survival rate for a single sensor (node) per day (time step).
        
        days(positive int): Represents the maximum number of days (time-steps) the network will be degraded.
        
    Returns: 
        end(int): Represents the last 't' (day) the function iterated over. This typically equals the time step when the
        network changes from connected to disconnected, however this can also represent the time step when only one node
        remains in the network, or the final time step if the network remained connected through each iteration of
        degradation.
        
    """
    H = network
    
    for t in range(1,days+1):
        if nx.is_connected(H) == True:
            
            if len(H.nodes) > 1:
                for i in set(H.nodes):
                    Fails = 100 - int(survivability)
                    RandomNum = random.randint(1,100)
                    if RandomNum > Fails:
                        pass
                    if RandomNum <= Fails:
                        new_nodes = H.remove_node(i)
                        for j in H.nodes:
                            if (i,j) in H.edges:
                                new_edges = H.remove_edge(i,j)
                                H.update(edges=new_edges, nodes=new_nodes)
            if len(H.nodes) <= 1:
                break
        
    
        if nx.is_connected(H) == False:
            break
    end = t  
    
    
    return end

def average_network_durability(network,survivability,days,replications):
    """
    This function replicates the network_degradation_test over a set number of replications in order to provide the user
    with the average number of days until the network becomes disconnected (fails). The user is also provided a histogram
    for the results of the replications, as well as the prediction interval for 95% of replications.
    
       
    Inputs:
        network(nx.Graph object): See network_degradation_test docstring
        
        survibability(int with range (0-100)): See network_degradation_test docstring
        
        days(positive int): See network_degradation_test docstring
        
        replications(positive int): The number of times the network_degradation_test function will be applied to the
        given network with its associated sensor survivability rate and number of days it can be degraded.
        
    Returns:
        mean(float): The average time step (day) the network becomes disconnected.
        
        lb(float): The lower bound of a 95% prediction interval.
        
        ub(float): The upper bound of a 95% prediction interval.
        
        plot: The histogram displaying the results of the function, placing each network_degradation_test results into 
        a bin. A 'kde' (Kernal Density Estimate) distribution curve is also provided.
    
    """
    N = network
    S = survivability
    D = days
    R = replications
    
    ave_fail_day = []

    original_network = network.copy()
    for r in range(1,R+1): #replications
        ave_fail_day.append(network_degradation_test(N.copy(),S,D))
        N = original_network
    
    mean = np.mean(ave_fail_day)
    lb = max(np.mean(ave_fail_day) - 1.96*(np.std(ave_fail_day)),0)
    ub = min(np.mean(ave_fail_day) + 1.96*(np.std(ave_fail_day)),days)
    plot = sns.histplot(data=ave_fail_day,x=ave_fail_day,y=None,hue=None,weights=None,stat='count',bins='auto',binwidth=1,binrange=None,discrete=None,
    cumulative=False,common_bins=True,common_norm=True,multiple='layer',element='bars',fill=True,shrink=1,kde=True,kde_kws=None,
    line_kws=None,thresh=0,pthresh=None,pmax=None,cbar=False,cbar_ax=None,cbar_kws=None,palette=None,hue_order=None,
    hue_norm=None,color=None,log_scale=None,legend=True,ax=None).set(title='Failure Distribution',xlabel='Time to Failure(Days)',ylabel='Number of Runs')
    plt.grid(visible=True, which='major', axis='y')
    
    figs = plt.gcf()
    
    return mean, lb, ub, figs

# The part that draws the MATPLOTLIB Graph into the GUI

In [6]:
"""
DOCSTRING NOTES: 
    The first function draws the graph on the sg Canvas object in PySimpleGUI.
    The second function deletes the MATPLOTLIB figure object.  
    This is so matplotlib doesn't draw over previously created objects
    
"""


def draw_figure(canvas, figure):
    """Draws the figure for plotting into the GUI
    
    Inputs:
        canvas (PySimpleGui Object): The canvas to draw to in PySimpleGui
        figure (The matplotlib figure): The figure image, taken from the OR functions in other parts of the program
        
    Returns:
        The drawn figure which displays in the GUI"""
    
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
    return figure_canvas_agg

def delete_figure_agg(figure_canvas_agg):
    """Deletes the current set of figures so that new figures can plot properly in the App
    
    Inputs: the current drawn figure held in memory
    Returns: None
    """
    
    figure_canvas_agg.get_tk_widget().forget()
    plt.close('all')
    


# Coding the Layout of the GUI

In [7]:
########################## Graphical User Interface ############################
"""Generally speaking, we are looking to construct a tabbed interface where each tab is organized by function.
    Within each tab, the left side will take inputs, and the right side will display outputs.
    As a design choice (tailored for the E-4 population), we will limit user response to list boxes
    and thus we prevent the user from making data input mistakes.
    """
######################### Defining GUI parameters, listboxes to use, lengthy text ################

#splash screen description
line1a = "This program is for a junior Intel Analyst who is maintaining the Sensor Network over a Named Area of Interest (NAI).  " 
line2a = "There are two use cases. The user either has an existing network that they can input via CSV and run reliability analysis on.  " 
line3a = "Or, they have an NAI that needs to be covered in the future.  " 
line4a = "They can then generate a hypothetical sensor network, deployed via airdrop or artillery.  "
line5a = "Then, they can run reliability analysis on it and experiment with the number of sensors and type deployed."
home_page_message = line1a + line2a + line3a + line4a + line5a

#General Instructions
line1b = "1. Click on the tab that describes what you want to do. " 
line2b = "2. Input your data (as applicable) or define your network parameters and click the button below the parameters. " 
line3b = "3. View the output.  If reliability analysis is desired, click the reliability analysis tab.  Your current network stays inputted into the program. "
line4b = "4. Input the parameters for reliability analysis and click the button below the parameters."
line5b = "5. Interpret the Analysis Graph and Report."
instructions_message = line1b + line2b + line3b + line4b + line5b
simple_instructions = "Right click anywhere to pull up application instructions.  Or click the help button on the top menu."

#GUI style theme
theme = "brownblue"

#Size of MATPLOTLIB graphic in GUI
figure_w, figure_h = 800, 800

# Lists for Combo Boxes in GUI
rangelist = ["5km", "10km", "15km"]
num_sensors = [25,50,75,100]
num_of_runs = [250,500,750,1000]
failure_rate = [.05, .10, .15, .20]
orientationlist = ['Horizontal', 'Slanted up', 'Slanted down','Vertical']

# Dictionaries to translate 'str' user input to the proper parameters
range_dictionary = {"5km": 5, "10km": 10,"15km": 15}
orientation_dictionary = {'Horizontal':[[0,0],[120,1]],
                          'Slanted up':[[1,1],[85,85]],
                          'Vertical': [[1,1],[5,120]],
                          'Slanted down': [[1,85],[85,2]]}


######################### Build the GUI Window ################

def make_window(theme):
    
    """
    This function proceeds in the following steps:
        1. Define the Menu Bar
        2. Define each tab.
        3. Within each tab, define the left side (input), right side(output) and then put them together.
        4. Define the Tab Groups
        5. Puts all the steps together and makes the window.
        
        Inputs:
            theme (sg.theme object): The thematic design choices of GUI Window colors and boxes
            
        Returns:
            window (sg.window object): The GUI window
            
        """
    
    #top of the program frame
    sg.theme(theme)
    menu_def = [['&Application', ['&Exit']],
                ['&Help', ['&About','&Sensors','&Instructions']] ]
    right_click_menu_def = [[], ['Display Instructions']]
   
    # Opening Tab
    start_left = [
            [sg.Text('Stochastically Placed Sensor Network Analyzer', expand_x=True, justification='center',
                font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Image("Network-PNG-Transparent.png", size=(450,450))]]
   
    start_right = [
            [sg.Text("About this Program", expand_x=True, justification='center',
                font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text(home_page_message, size=(40,11),
                font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text("Access Instructions Anywhere", size=(40, 1), justification='center',
                font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text(simple_instructions, size=(40,4),
                font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)]]
   
    start_layout = [
            [sg.Col(start_left), sg.VerticalSeparator(),sg.Col(start_right)]]

   
    #Data Input Tab
    data_input_left = [
            [sg.Text('Existing Network Input', expand_x=True, justification='center',
                font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text("Select your .CSV File: ", key="-FileInput1-"), sg.Input(size=(23, 1)), sg.FileBrowse()],
            [sg.Text("Select the sensor Comms Range: ", key="-CommsRange1-"), sg.Combo(rangelist)],
            [sg.Button("Generate and Run Analysis", key="-UploadAndRunAnalysis1-")]]
   
    data_input_right = [
            [sg.Text('Data Display', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Canvas(size=(600, 600), key='-CANVAS1a-')],
            [sg.Canvas(size=(600, 600), key='-CANVAS1b-')]]
   
    data_input_layout = [
            [sg.Col(data_input_left,p=10), sg.VerticalSeparator(),sg.Col(data_input_right,p=10)]]
   
       
    #Generate Random Network Airborne Tab
    gen_rand_left_airborne = [
            [sg.Text('Network Parameters for Random Generation', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text("Select the Deployment Orientation: ", key="-StartPoint2-"), sg.Combo(orientationlist)],
            [sg.Text("Select the Number of Sensors: ", key="-NumberSensors2-"), sg.Combo(num_sensors)],
            [sg.Text("Select the sensor Comms Range: ", key="-CommsRange2-"), sg.Combo(rangelist)],
            [sg.Button("Generate and Run Analysis", key="-AirborneAnalysis2-")]]
   
    gen_rand_right_airborne = [
            [sg.Text('Data Display', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS2a-')],
            [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS2b-')]]
   
    gen_rand_network_layout_airborne = [
            [sg.Col(gen_rand_left_airborne,p=10), sg.VerticalSeparator(),sg.Col(gen_rand_right_airborne,p=10)]]
   
       
    #Generate Random Network Artillery Tab
    gen_rand_left_artillery = [
            [sg.Text('Network Parameters for Random Generation', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text("Select the Number of Sensors: ", key="-NumberSensors3-"), sg.Combo(num_sensors)],
            [sg.Text("Select the sensor Comms Range: ", key="-CommsRange3-"), sg.Combo(rangelist)],
            [sg.Button("Generate and Run Analysis", key= "-ArtilleryAnalysis3-")]]
   
    gen_rand_right_artillery = [
            [sg.Text('Data Display', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS3a-')],     #where the MATPLOTLIB draws to
            [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS3b-')]]
   
    gen_rand_network_layout_artillery = [
            [sg.Col(gen_rand_left_artillery,p=10), sg.VerticalSeparator(),sg.Col(gen_rand_right_artillery,p=10)]]
   
   
    #Reliability Analysis Tab
    reliability_left = [
            [sg.Text('Parameters for Reliability Analysis', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Text("Select the Number of Runs: ", key="-NumberRuns4-"), sg.Combo(num_of_runs)],
            [sg.Text("Select the Failure Rate per Day: ", key="-FailureRate4-"), sg.Combo(failure_rate)],
            [sg.Text("Select the Maximum Number of Days to simulate: ", key="-MaxDays4-"), sg.Combo(num_sensors)],
            [sg.Button("Run Analysis", key= "-ReliabilityAnalysis4-")]]
           
    reliability_right = [
            [sg.Text('Network Dis-Connectivity Histogram', expand_x=True, justification='center',
                 font=("Helvetica", 16), relief=sg.RELIEF_RIDGE)],
            [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS4a-')], #where the MATPLOTLIB draws to
            [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS4b-')],
            [sg.Button("Empty Graph?", key= "-NoReliabilityGraph-")]]
   
    reliability_analysis_layout = [
            [sg.Col(reliability_left,p=10), sg.VerticalSeparator(),sg.Col(reliability_right,p=10)]]
   
   
    #Overall Layout
    layout = [
            [sg.MenubarCustom(menu_def, key='-MENU-', font='Courier 15', tearoff=True)]]
   
    layout +=[[sg.TabGroup([[  sg.Tab('Start', start_layout,
                                      key="-StartTab-"),
                               sg.Tab('Existing Network Input', data_input_layout,
                                      key="-DataInputTab-"),
                               sg.Tab('Generate Random Network AIRBORNE', gen_rand_network_layout_airborne,
                                      key="-AirborneTab-"),
                               sg.Tab('Generate Random Network ARTILLERY', gen_rand_network_layout_artillery,
                                      key="-ArtilleryTab-"),
                               sg.Tab('Reliability Analysis', reliability_analysis_layout,
                                      key="-ReliabilityTab-"),
                               ]], key='-TAB GROUP-', expand_x=True, expand_y=True)]]
   

    window = sg.Window('Sensor Network Analyzer', layout, grab_anywhere=True, margins=(0,0), right_click_menu=right_click_menu_def,
                       use_custom_titlebar=True, finalize=True, size=(1000,800), keep_on_top=False, resizable=True)

   
    return window

# Make it all work together: The main program

In [ ]:
########################## Tie it all together as a program #########

    
def main():
    """ This function creates the gui window and pulls all the previously defined functions created previously to 
    operate the user interface. Allowing the DSS to function.
    
    Input:
        make_window(sg.theme())(function): Allows gui to open and operate
        functions defined above (functions): Internal functions for the DSS
    Returns: 
        Functioning DSS
    """
    
    #initialize output variables here
    figure_canvas_agg = None
    figure_canvas_agg2 = None
    window = make_window(sg.theme())
    
    while True:
        event, values = window.read()
        

        if event in (None, 'Exit'):
            print("[LOG] Clicked Exit!")
            break
            
        elif event == "-UploadAndRunAnalysis1-":
            
            # ** IMPORTANT ** Clean up previous drawing before drawing again
            if figure_canvas_agg:
                delete_figure_agg(figure_canvas_agg) 
            if figure_canvas_agg2:
                delete_figure_agg(figure_canvas_agg2) 
            
            try:
                file = values[2]
                comms_range = range_dictionary[values[3]]
                
                G, X, Y, figure_canvas_agg = network_from_xy_csv(file,comms_range)
                figure_canvas_agg2 = text_report(G)
            
                figure_canvas_agg = draw_figure(window['-CANVAS1a-'].TKCanvas, figure_canvas_agg)
                figure_canvas_agg2 = draw_figure(window['-CANVAS1b-'].TKCanvas, figure_canvas_agg2)
            
            except KeyError:
                sg.popup("Error: Check to make sure all your parameters are filled out", keep_on_top=True)
                pass
            except:
                sg.popup("Error: Check to make you you selected the correct CSV file.",
                         "The real-world sensor network location report is produced in the correct x-coord, y-coord format",
                         "So it will work if you select the correct CSV.",
                         keep_on_top=True)
                pass
        
        elif event == "-AirborneAnalysis2-":
            if figure_canvas_agg:
                delete_figure_agg(figure_canvas_agg) 
            if figure_canvas_agg2:
                delete_figure_agg(figure_canvas_agg2) 
                
            try:
                left_point = orientation_dictionary[values[5]][0] 
                right_point = orientation_dictionary[values[5]][1]
                num_sensors = values[6]
                comms_range = range_dictionary[values[7]]

                G, X, Y, figure_canvas_agg = generate_network_from_two_points(left_point, right_point,
                                                                              comms_range, max_num_sensors=num_sensors)
                figure_canvas_agg2 = text_report(G)

                figure_canvas_agg = draw_figure(window['-CANVAS2a-'].TKCanvas, figure_canvas_agg)
                figure_canvas_agg2 = draw_figure(window['-CANVAS2b-'].TKCanvas, figure_canvas_agg2)
            
            except:
                sg.popup("Error: Check to make sure all your parameters are filled out", keep_on_top=True)
                pass
            
        elif event == "-ArtilleryAnalysis3-":
            if figure_canvas_agg:
                delete_figure_agg(figure_canvas_agg) 
            if figure_canvas_agg2:
                delete_figure_agg(figure_canvas_agg2) 
            
            try:
                nodes = values[9]
                comms_range = range_dictionary[values[10]]

                G, X, Y, figure_canvas_agg = generate_center_out_network(nodes, comms_range)
                figure_canvas_agg2 = text_report(G)

                figure_canvas_agg = draw_figure(window['-CANVAS3a-'].TKCanvas, figure_canvas_agg)
                figure_canvas_agg2 = draw_figure(window['-CANVAS3b-'].TKCanvas, figure_canvas_agg2)
            
            except:
                sg.popup("Error: Check to make sure all your parameters are filled out", keep_on_top=True)
                pass
            
        elif event == "-ReliabilityAnalysis4-":
            if figure_canvas_agg:
                delete_figure_agg(figure_canvas_agg) 
            if figure_canvas_agg2:
                delete_figure_agg(figure_canvas_agg2)
            
            try:
                network = G
                survivability = 100 - values[13]*100
                days = values[14]
                replications = values[12]

                mean, lower_bound, upper_bound, figure_canvas_agg = average_network_durability(network,survivability,days,replications)
                figure_canvas_agg2 = text_report_reliability(G, mean, lower_bound, upper_bound)               

                figure_canvas_agg = draw_figure(window['-CANVAS4a-'].TKCanvas, figure_canvas_agg)               
                figure_canvas_agg2 = draw_figure(window['-CANVAS4b-'].TKCanvas, figure_canvas_agg2)
                        
            except UnboundLocalError:
                sg.popup("Error: There is no network to analyze.",
                         "Upload or generate a network first in the appropriate tab.", keep_on_top=True)
                pass    
            
            except (KeyError,TypeError):
                sg.popup("Error: Check to make sure all your parameters are filled out", keep_on_top=True)
                pass    
                
        elif event == 'Display Instructions':
            sg.popup("Instructions:",
                     line1b, 
                     line2b, 
                     line3b, 
                     line4b, 
                     line5b, keep_on_top=True)
        elif event == 'About':
            sg.popup("About:",
                     line1a,
                     line2a,
                     line3a,
                     line4a,
                     line5a, keep_on_top=True)
            
        elif event == 'Sensors':
            sg.popup("Sensors:",
                     "Each sensor in the US inventory comes in one of three types",
                     "The Mk1 has a comms radius of 5km and a scan radius of 2.5km",
                     "The Mk2 has a comms radius of 10km and a scan radius of 5km",
                     "The Mk3 has a comms radius of 15km and a scan radius of 7.5km",
                     "Because the sensor scan radius is one-half the comms radius, the enemy cannot pass undectected between any connected sensors.  "
                     "We additionally leverage this fact to define network reliability.  "
                     "The network goes from reliable to unreliable when the network is no longer connected.  "
                     "This is the moment that enemy can maneuver undetected THROUGH our sensor network.", keep_on_top=True)
        elif event == '-NoReliabilityGraph-':
            sg.popup("The inputed graph is already not connected.",
                     "Please generate another graph before attempting to run an analysis.",
                    keep_on_top=True)
        
        elif event == 'Instructions':
            sg.popup("Instructions:",
                     line1b,
                     line2b,
                     line3b,
                     line4b,
                     line5b, keep_on_top=True)
        
        
    window.close()
    #exit(0)    #for later in case we build a .py script from this whole project.

main()

Total sensors deployed:  8
Surviving Nodes:  8
Total sensors deployed:  100
Surviving Nodes:  77
Total sensors deployed:  100
Surviving Nodes:  100
Total sensors deployed:  100
Surviving Nodes:  100
Total sensors deployed:  50
Surviving Nodes:  43
Total sensors deployed:  75
